# Libraries

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Parameters

In [ ]:
N = 1_000_000
D = 4.0             # Infection duration (1/gamma)
gamma = 1.0 / D     # Recovery rate
delta = 1.0 / 5.0   # Incubation rate (1/delta)
rho = 1/9           # Fatality delay

R_0_start, k, x0, R_0_end = 5.0, 0.5, 50, 0.5

def beta(t):
    r0_t = (R_0_start - R_0_end) / (1 + np.exp(-k * (-t + x0))) + R_0_end
    return r0_t * gamma

def heaviside(t, start_day):
    return 1.0 if t >= start_day else 0.0

# Single Dose with 100% immunity

In [ ]:
def deriv_phase1(y, t, N, gamma, delta, rho):
    S, E, I, R, D_comp, V = y

    vac_rate = 0.01
    vac_start = 30
    
    # Dynamic Variables
    alpha = get_fatality_rate(I, N)
    lam = beta(t) * I / N             # Force of infection
    
    # Vaccination Logic (Phase 1: Perfect Sink)
    vaccination_flow = vac_rate * S * heaviside(t, vac_start)
    
    # Differential Equations
    dSdt = -lam * S - vaccination_flow
    dEdt = lam * S - delta * E
    dIdt = delta * E - (1 - alpha) * gamma * I - alpha * rho * I
    dRdt = (1 - alpha) * gamma * I
    dDdt = alpha * rho * I
    
    # V equation: purely gaining from S
    dVdt = vaccination_flow 
    
    return dSdt, dEdt, dIdt, dRdt, dDdt, dVdt